In [4]:
# creating spectrograms from all the files, and saving split labelled versions to disk ready for machine learning
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt

import os
import sys
import cPickle as pickle
import numpy as np
import time
import random 
import yaml

from scipy.ndimage.interpolation import zoom

#from data_helpers import load_annotations

import nolearn
import nolearn.lasagne
import lasagne.layers

from lasagne.layers import InputLayer, DimshuffleLayer
from lasagne.layers import DenseLayer
from lasagne.layers import NonlinearityLayer
from lasagne.layers import DropoutLayer
from lasagne.layers import Pool2DLayer as PoolLayer
from lasagne.layers.dnn import Conv2DDNNLayer as ConvLayer
from lasagne.nonlinearities import softmax, very_leaky_rectify as vlr
import theano

base = '/media/michael/Seagate/engage/alison_data/golden_set/'
annotation_pkl_dir = base + 'extracted/annotations/'
spec_pkl_dir = base + 'extracted/specs/'
log_dir = base + 'ml_runs/'

Using gpu device 0: GeForce GTX 770 (CNMeM is disabled, cuDNN 5004)
/home/michael/anaconda/lib/python2.7/site-packages/Theano-0.9.0.dev1-py2.7.egg/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [131]:
HWW = 15
SPEC_HEIGHT = 330
LEARN_LOG = True
DO_AUGMENTATION = True
DO_BATCH_NORM = True

In [132]:
# loading splits
splits = yaml.load(open(base + 'splits/folds.yaml'))
train = splits[0] + splits[1]
test = splits[2]
                
# load data and make list of specsamplers
train_data = [[], []]
test_data = [[], []]

for fname in os.listdir(spec_pkl_dir):
    
    # load spectrogram and annotations
    spec = pickle.load(open(spec_pkl_dir + fname))[:SPEC_HEIGHT, :]
    annots, wav, sample_rate = pickle.load(open(annotation_pkl_dir + fname))
        
    # reshape annotations
    for classname in annots:
        factor = float(spec.shape[1]) / annots[classname].shape[0]
        annots[classname] = zoom(annots[classname], factor)
        
    # create sampler
    if not LEARN_LOG:
        spec = np.log(0.001 + spec)
        spec = spec - np.median(spec, axis=1, keepdims=True)

    if fname in train:
        train_data[0].append(spec)
        train_data[1].append(annots['anthrop'])
    elif fname in test:
        test_data[0].append(spec)
        test_data[1].append(annots['anthrop'])
        
print len(test_data[0])
print len(train_data[0])

14
26


In [133]:
class AllSpecs(object):
    
    def __init__(self, specs, labels, hww, do_aug, learn_log):
        self.do_aug = do_aug
        self.learn_log = learn_log
        self.hww = hww
        
        blank_spec = np.zeros((specs[0].shape[0], hww))
        self.specs = np.hstack([blank_spec] + specs + [blank_spec])[None, ...]
        
        blank_label = np.zeros(hww) - 1
        self.labels = np.hstack([blank_label] + labels + [blank_label])
        
        which_spec = [ii * np.ones_like(lab) for ii, lab in enumerate(labels)]
        self.which_spec = np.hstack([blank_label] + which_spec + [blank_label])
        
        if learn_log:
            self.medians = np.zeros((len(specs), specs[0].shape[0], hww*2))
            for idx, spec in enumerate(specs):
                self.medians[idx] = np.median(spec, axis=1, keepdims=True)

    def sample(self, num_per_class, seed=None):
        
        tic = time.time()
        num_samples = num_per_class * 2
        channels = self.specs.shape[0]
        height = self.specs.shape[1]
        
        if seed is not None:
            np.random.seed(seed)

        X = np.zeros((num_samples, channels, height, self.hww*2), np.float32)
        y = np.zeros(num_samples) * np.nan
        if self.learn_log:
            X_medians = np.zeros((num_samples, channels, height, self.hww*2), np.float32)
        count = 0
        
        for cls in [0, 1]:
            possible_locs = np.where(self.labels==cls)[0]

            if len(possible_locs) >= num_per_class:
                sampled_locs = np.random.choice(possible_locs, num_per_class, replace=False)

                for loc in sampled_locs:
                    X[count] = self.specs[:, :, (loc-self.hww):(loc+self.hww)]
                    y[count] = cls
                    if self.learn_log:
                        which = self.which_spec[loc]
                        X_medians[count] = self.medians[which]
                    count += 1

        # doing augmentation
        if self.do_aug:
            if self.learn_log:
                X *= (1.0 + np.random.randn(num_samples, 1, 1, 1) * 0.1)
            else:
                X *= (1.0 + np.random.randn(num_samples, 1, 1, 1) * 0.1)
                X += np.random.randn(num_samples, 1, 1, 1) * 0.05

        if self.learn_log:
            xb = {'input': X.astype(np.float32), 'input_med': X_medians.astype(np.float32)}
            return xb, y.astype(np.int32)
            
        else:
            # remove ones we couldn't get
            return X.astype(np.float32), y.astype(np.int32)        

In [134]:
train_sampler = AllSpecs(train_data[0], train_data[1], HWW, DO_AUGMENTATION, LEARN_LOG)
test_sampler = AllSpecs(test_data[0], test_data[1], HWW, False, LEARN_LOG)

In [135]:
class MyBatch(nolearn.lasagne.BatchIterator):
    def __iter__(self):
        bs = self.batch_size
        for _ in range(32):
            yield self.X.sample(bs)

            
class MyBatchTest(nolearn.lasagne.BatchIterator):
    def __iter__(self):
        bs = self.batch_size
        for idx in range(128):
            yield self.X.sample(bs, seed=idx)
                

mb = MyBatch(20)
mb.X = train_sampler

for count, (xx, yy) in enumerate(mb):
    if LEARN_LOG:
        print xx['input'].min(), xx['input_med'].shape, yy.sum()
    else:
        print xx.shape, yy.shape, yy.sum(), xx.sum()


3.04916e-05 (40, 1, 330, 30) 20
1.23559e-05 (40, 1, 330, 30) 20
3.78513e-05 (40, 1, 330, 30) 20
1.0475e-05 (40, 1, 330, 30) 20
1.96681e-05 (40, 1, 330, 30) 20
2.07971e-05 (40, 1, 330, 30) 20
1.38334e-06 (40, 1, 330, 30) 20
3.23498e-05 (40, 1, 330, 30) 20
5.51634e-05 (40, 1, 330, 30) 20
4.58004e-05 (40, 1, 330, 30) 20
2.28302e-05 (40, 1, 330, 30) 20
1.79445e-05 (40, 1, 330, 30) 20
1.13151e-05 (40, 1, 330, 30) 20
1.40399e-05 (40, 1, 330, 30) 20
3.97452e-05 (40, 1, 330, 30) 20
2.63449e-05 (40, 1, 330, 30) 20
1.72209e-05 (40, 1, 330, 30) 20
3.13341e-05 (40, 1, 330, 30) 20
3.36542e-05 (40, 1, 330, 30) 20
2.20208e-05 (40, 1, 330, 30) 20
2.21843e-05 (40, 1, 330, 30) 20
9.74864e-06 (40, 1, 330, 30) 20
3.2903e-05 (40, 1, 330, 30) 20
3.68806e-05 (40, 1, 330, 30) 20
1.97176e-05 (40, 1, 330, 30) 20
2.8982e-05 (40, 1, 330, 30) 20
8.49842e-06 (40, 1, 330, 30) 20
1.74721e-05 (40, 1, 330, 30) 20
2.50631e-05 (40, 1, 330, 30) 20
5.63585e-05 (40, 1, 330, 30) 20
1.23977e-05 (40, 1, 330, 30) 20
6.20494e-06

/home/michael/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:49: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [136]:
class MyTrainSplit(nolearn.lasagne.TrainSplit):
    # custom data split
    def __call__(self, data, Yb, net):
        return train_sampler, test_sampler, None, None

In [149]:
# from lasagne.nonlinearities import elu as vlr
from lasagne.nonlinearities import softmax, elu as vlr
from lasagne.layers import batch_norm, ElemwiseSumLayer, ExpressionLayer, DimshuffleLayer
from helpers import Log1Plus, ForgetSizeLayer
import theano.tensor as T

NUM_FILTERS = 32

if not DO_BATCH_NORM:
    batch_norm = lambda x: x
net = {}

# main input layer, then logged
net['input'] = InputLayer((None, 1, SPEC_HEIGHT, HWW*2), name='input')

if LEARN_LOG:
    off = lasagne.init.Constant(0.5)
    mult = lasagne.init.Constant(1.0)

    net['input_logged'] = Log1Plus(net['input'], off, mult)

    # logging the median and multiplying by -1
    net['input_med'] = InputLayer((None, 1, SPEC_HEIGHT, HWW*2), name='input_med')
    net['med_logged'] = Log1Plus(net['input_med'], off=net['input_logged'].off, mult=net['input_logged'].mult)
    net['med_logged'] = ExpressionLayer(net['med_logged'], lambda X: -X)

    # summing the logged input with the negative logged median
    net['input'] = ElemwiseSumLayer((net['input_logged'], net['med_logged']))

net['conv1_1'] = batch_norm(
    ConvLayer(net['input'], NUM_FILTERS, (spec.shape[0] - 1, 6), nonlinearity=vlr))
net['pool1'] = PoolLayer(net['conv1_1'], pool_size=(2, 2), stride=(2, 2), mode='max')
net['pool1'] = DropoutLayer(net['pool1'], p=0.5)
net['conv1_2'] = batch_norm(ConvLayer(net['pool1'], NUM_FILTERS, (1, 3), nonlinearity=vlr))
# net['pool2'] = PoolLayer(net['conv1_2'], pool_size=(1, 2), stride=(1, 1))
net['pool2'] = DropoutLayer(net['conv1_2'], p=0.5)

net['fc6'] = batch_norm(DenseLayer(net['pool2'], num_units=64, nonlinearity=vlr))
net['fc6'] = DropoutLayer(net['fc6'], p=0.5)
net['fc7'] = batch_norm(DenseLayer(net['fc6'], num_units=64, nonlinearity=vlr))
net['fc7'] = DropoutLayer(net['fc7'], p=0.5)
net['fc8'] = DenseLayer(net['fc7'], num_units=2, nonlinearity=None)
net['prob'] = NonlinearityLayer(net['fc8'], softmax)

net = nolearn.lasagne.NeuralNet(
    layers=net['prob'],
    max_epochs=500,
    update=lasagne.updates.adam,
    update_learning_rate=0.0001,
#     update_momentum=0.975,
    verbose=1,
    batch_iterator_train=MyBatch(128),
    batch_iterator_test=MyBatchTest(128),
    train_split=MyTrainSplit(None),
    check_input=False
)

In [ ]:
net.fit(None, None)

# Neural Network with 91332 learnable parameters

## Layer information

  #  name       size
---  ---------  --------
  0  input      1x330x30
  1             1x330x30
  2  input_med  1x330x30
  3             1x330x30
  4             1x330x30
  5             1x330x30
  6             32x2x25
  7             32x2x25
  8             32x2x25
  9             32x1x12
 10             32x1x12
 11             32x1x10
 12             32x1x10
 13             32x1x10
 14             32x1x10
 15             64
 16             64
 17             64
 18             64
 19             64
 20             64
 21             64
 22             64
 23             2
 24             2

  epoch    train loss    valid loss    train/val    valid acc  dur
-------  ------------  ------------  -----------  -----------  -----
      1       1.05796       0.53622      1.97299      0.71683  6.03s
      2       0.86872       0.47349      1.83473      0.78467  6.05s
      3       0.74832       0.43822      1.70763     

In [96]:
print net.layers_['log1plus1'].off.get_value()
print net.layers_['log1plus3'].off.get_value()
print net.layers_['log1plus1'].mult.get_value()
print net.layers_['log1plus3'].mult.get_value()

[ 0.46714729]
[ 0.46714729]
[ 1.03194082]
[ 1.03194082]


In [ ]:
def plot_loss(net):
    train_loss = [row['train_loss'] for row in net.train_history_]
    valid_loss = [row['valid_loss'] for row in net.train_history_]
    plt.plot(train_loss, label='train loss')
    plt.plot(valid_loss, label='valid loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='best')
    return plt
plot_loss(net)

In [ ]:
y = [0.197185359589, 0.518698018591, 0.581381482387, 0.664551736791, 0.699983182485]
x = [100, 500, 1000, 2500, 5000]
